In [1]:
import numpy as np

import ot 
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from OTUnawareFairRegressor import OTUnawareFairRegressor 
from data_extraction_script import read_adult_dataset
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from sklearn.metrics import mean_squared_error

In [ ]:
from ucimlrepo import fetch_ucirepo 


def read_adult_dataset():
    # fetch dataset 
    adult = fetch_ucirepo(id=2) 
    
    # data (as pandas dataframes) 
    X = adult.data.features
    S = X['sex'] #sensitive attribute
    X = X.drop(columns=['sex'])
    y = adult.data.targets 
    
    #metadata 
    #print(adult.metadata) 
    
    # variable information 
    #print(adult.variables) 

    return X,S,y

read_adult_dataset()